In [3]:
pip install threading

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement threading (from versions: none)
ERROR: No matching distribution found for threading


In [1]:
pip install opencv-python face_recognition pillow playsound


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tkinter as tk
import cv2
import numpy as np
import face_recognition
from datetime import datetime
from PIL import Image, ImageTk
from playsound import playsound
import threading
import time

# Initialize main window
root = tk.Tk()
root.title("Face Recognition Attendance System")
root.geometry("800x500")

# Global variables for capturing webcam feed and stopping it
cap = None
running = False

# Load the images for recognition
image_mohith_path = r"C:\Users\Mohith V Krishnam\OneDrive\Desktop\Mini Project\mohith.jpg"
image_ashutosh_path = r"C:\Users\Mohith V Krishnam\OneDrive\Desktop\Mini Project\WhatsApp Image 2024-10-15 at 11.10.08_29080e4c.jpg"
image_krishna_path = r"C:\Users\Mohith V Krishnam\OneDrive\Desktop\Mini Project\WhatsApp Image 2024-10-18 at 18.21.23_45040acd.jpg"
image_syed_nadeem_path = r"C:\Users\Mohith V Krishnam\OneDrive\Desktop\Mini Project\WhatsApp Image 2024-10-20 at 11.42.23_413aa4e1.jpg"

image_mohith = cv2.imread(image_mohith_path)
image_ashutosh = cv2.imread(image_ashutosh_path)
image_krishna = cv2.imread(image_krishna_path)
image_syed_nadeem = cv2.imread(image_syed_nadeem_path)

images = [image_mohith, image_ashutosh, image_krishna, image_syed_nadeem]
classNames = ["Mohith", "Ashutosh Kumar", "Krishna", "Syed Nadeem Ahmed"]

# User details dictionary
user_details = {
    "Mohith": {
        "age": 20,
        "dob": "11/01/2004"
    },
    "Ashutosh Kumar": {
        "age": 22,
        "dob": "12/05/2002"
    },
    "Krishna": {
        "age": 21,
        "dob": "05/10/2003"
    },
    "Syed Nadeem Ahmed": {
        "age": 23,
        "dob": "01/11/2000"
    }
}

# Path for beep sound
beep_sound_path = r"C:\Users\Mohith V Krishnam\OneDrive\Desktop\Mini Project\race-start-beeps-125125.mp3"

# Function to encode faces
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

# Encode known faces
encodeListKnown = findEncodings(images)
print('Encoding Complete')

# Function to mark attendance
def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = [line.split(',')[0] for line in myDataList]
        
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')
            update_log(f'{name} marked present at {dtString}')

# Function to update attendance log in the GUI
def update_log(message):
    attendance_log.insert(tk.END, message)
    attendance_log.yview(tk.END)

# Function to start the webcam
def start_webcam():
    global cap, running
    if cap is None:
        cap = cv2.VideoCapture(0)
        running = True
        status_label.config(text="Webcam Status: Running", fg="green")
        threading.Thread(target=show_frame, daemon=True).start()  # Start frame processing in a thread

def stop_webcam():
    global cap, running
    if running:
        running = False
        cap.release()
        cv2.destroyAllWindows()
        status_label.config(text="Webcam Status: Stopped", fg="red")
        video_label.config(image="")

def quit_app():
    stop_webcam()
    root.quit()

def beep_for_unknown():
    playsound(beep_sound_path)

def show_frame():
    global cap, running
    frame_skip_count = 0  # Initialize frame skip counter
    while running:
        success, img = cap.read()
        if success:
            frame_skip_count += 1
            # Skip frames to increase the processing rate
            if frame_skip_count % 2 == 0:  # Process every second frame
                imgS = cv2.resize(img, (0, 0), None, 0.5, 0.5)
                imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

                facesCurFrame = face_recognition.face_locations(imgS)
                encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

                recognized = False
                name = "Unknown"

                for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
                    matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
                    faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

                    matchIndex = np.argmin(faceDis)

                    if matches[matchIndex] and faceDis[matchIndex] < 0.6:
                        name = classNames[matchIndex].upper()
                        print(f"Recognized: {name}")  # Debugging output
                        display_user_info(name)
                        markAttendance(name)
                        recognized = True
                        color = (0, 255, 0)  # Green for known
                    else:
                        if not recognized:  # Only beep if no known face was recognized
                            beep_for_unknown()  # Trigger beep for unknown
                        color = (0, 0, 255)  # Red for unknown

                    y1, x2, y2, x1 = faceLoc
                    y1, x2, y2, x1 = y1 * 2, x2 * 2, y2 * 2, x1 * 2

                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                    cv2.rectangle(img, (x1, y2 - 35), (x2, y2), color, cv2.FILLED)
                    cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(img)
                imgtk = ImageTk.PhotoImage(image=img)

                video_label.config(image=imgtk)
                video_label.imgtk = imgtk

                if not recognized:
                    clear_user_info()

        # Add a small delay to help with the frame rate
        time.sleep(0.01)  # Adjust this value to control the frame rate

# Function to display user information
def display_user_info(name):
    if name in user_details:
        age = user_details[name]["age"]
        dob = user_details[name]["dob"]
        info_text = f"Name: {name}\nAge: {age}\nDate of Birth: {dob}"
        info_label.config(text=info_text)
        print(f"Displaying info: {info_text}")  # Debugging output
    else:
        print(f"No details found for {name}")  # Debugging output

# Function to clear user information when face is unknown
def clear_user_info():
    info_label.config(text="")
    print("User info cleared")  # Debugging output

# GUI Layout
title_label = tk.Label(root, text="Face Recognition Attendance System", font=("Helvetica", 18, "bold"), bg="#4CAF50", fg="white")
title_label.pack(fill=tk.X, pady=10)

main_frame = tk.Frame(root)
main_frame.pack(pady=10)

# Video feed label
video_label = tk.Label(main_frame, bg="black")
video_label.pack(side=tk.LEFT)

# User Information Frame
info_frame = tk.Frame(main_frame, bg="#f0f0f0", padx=10, pady=10)
info_frame.pack(side=tk.LEFT)

# Information Label for Name, Age, and DOB
info_label = tk.Label(info_frame, text="", font=("Helvetica", 14), bg="#f0f0f0", justify=tk.LEFT)
info_label.pack(pady=20)

status_label = tk.Label(root, text="Webcam Status: Stopped", font=("Helvetica", 14), fg="red")
status_label.pack(pady=5)

log_frame = tk.Frame(root)
log_frame.pack(pady=10)

log_scrollbar = tk.Scrollbar(log_frame)
log_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

attendance_log = tk.Listbox(log_frame, height=8, width=60, font=("Helvetica", 12), yscrollcommand=log_scrollbar.set)
attendance_log.pack(side=tk.LEFT, fill=tk.BOTH)
log_scrollbar.config(command=attendance_log.yview)

button_frame = tk.Frame(root)
button_frame.pack(pady=20)

btn_start = tk.Button(button_frame, text="Start Webcam", command=start_webcam, width=15, bg="#4CAF50", fg="white", font=("Helvetica", 12))
btn_start.grid(row=0, column=0, padx=5)

btn_stop = tk.Button(button_frame, text="Stop Webcam", command=stop_webcam, width=15, bg="#FF9800", fg="white", font=("Helvetica", 12))
btn_stop.grid(row=0, column=1, padx=5)

btn_quit = tk.Button(button_frame, text="Quit", command=quit_app, width=15, bg="#F44336", fg="white", font=("Helvetica", 12))
btn_quit.grid(row=0, column=2, padx=5)

def on_closing():
    stop_webcam()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)

root.mainloop()